In [1]:
# basic set of imports
import re, operator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler

from itertools import chain


from IPython.display import display, HTML
import datetime, pickle
import time
import tables
%matplotlib inline

import ml_metrics as metrics
import operator
import random

In [2]:
filename = "destinations"
destinations = pd.read_csv(r"..\expedia_data\{0}.csv".format(filename))


In [3]:
filename = 'test_as_hdf'
test = pd.read_hdf(r"..\expedia_data\{0}.h5".format(filename))
test.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2528243 entries, 0 to 2528242
Data columns (total 22 columns):
id                           uint32
date_time                    datetime64[ns]
site_name                    uint8
posa_continent               uint8
user_location_country        uint8
user_location_region         uint16
user_location_city           uint16
orig_destination_distance    float64
user_id                      uint32
is_mobile                    uint8
is_package                   uint8
channel                      uint8
srch_ci                      datetime64[ns]
srch_co                      datetime64[ns]
srch_adults_cnt              uint8
srch_children_cnt            uint8
srch_rm_cnt                  uint8
srch_destination_id          uint32
srch_destination_type_id     uint8
hotel_continent              uint8
hotel_country                uint8
hotel_market                 uint16
dtypes: datetime64[ns](3), float64(1), uint16(3), uint32(3), uint8(12)
memory usag

In [2]:
filename = 'train_as_hdf'
train = pd.read_hdf(r"..\expedia_data\{0}.h5".format(filename))
train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37670293 entries, 0 to 37670292
Data columns (total 24 columns):
date_time                    datetime64[ns]
site_name                    uint8
posa_continent               uint8
user_location_country        uint8
user_location_region         uint16
user_location_city           uint16
orig_destination_distance    float64
user_id                      uint32
is_mobile                    uint8
is_package                   uint8
channel                      uint8
srch_ci                      datetime64[ns]
srch_co                      datetime64[ns]
srch_adults_cnt              uint8
srch_children_cnt            uint8
srch_rm_cnt                  uint8
srch_destination_id          uint32
srch_destination_type_id     uint8
is_booking                   uint8
cnt                          uint16
hotel_continent              uint8
hotel_country                uint8
hotel_market                 uint16
hotel_cluster                uint8
dtypes: da

In [8]:
for col in train.columns:
    print(col, train[col].isnull().values.any() )

date_time False
site_name False
posa_continent False
user_location_country False
user_location_region False
user_location_city False
orig_destination_distance True
user_id False
is_mobile False
is_package False
channel False
srch_ci True
srch_co True
srch_adults_cnt False
srch_children_cnt False
srch_rm_cnt False
srch_destination_id False
srch_destination_type_id False
is_booking False
cnt False
hotel_continent False
hotel_country False
hotel_market False
hotel_cluster False


In [66]:
import gc
gc.collect()

0

In [6]:
# train["hotel_cluster"].value_counts()


91    1043720
41     772743
48     754033
64     704734
65     670960
5      620194
98     589178
59     570291
42     551605
21     550092
70     545572
18     545284
83     534132
46     534038
25     530591
62     518809
95     509266
28     507016
68     503797
82     503755
37     496061
50     489892
30     489287
9      488328
58     483253
97     479446
16     477868
72     457463
1      452694
99     444887
       ...   
19     282893
84     278264
66     273505
38     269246
87     260398
23     259233
12     259022
31     257587
67     255946
43     253578
7      252447
54     250745
92     244343
89     243560
45     241408
49     240124
3      225250
80     220218
60     217919
71     216054
93     214293
86     209054
14     192299
75     165226
24     164127
35     139122
53     134812
88     107784
27     105040
74      48355
Name: hotel_cluster, dtype: int64

In [7]:
# train.shape

(37670293, 24)

In [8]:
# test.shape


(2528243, 22)

In [9]:
test_ids = set(test.user_id.unique())

In [10]:
train_ids = set(train.user_id.unique())

In [11]:
intersection_count = len(test_ids & train_ids)
intersection_count == len(test_ids)

True

In [12]:
# we're going to do this more systematically later on
# train["year"] = train["date_time"].dt.year
# train["month"] = train["date_time"].dt.month

In [13]:
# train.columns

Index(['date_time', 'site_name', 'posa_continent', 'user_location_country',
       'user_location_region', 'user_location_city',
       'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
       'channel', 'srch_ci', 'srch_co', 'srch_adults_cnt', 'srch_children_cnt',
       'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id',
       'is_booking', 'cnt', 'hotel_continent', 'hotel_country', 'hotel_market',
       'hotel_cluster', 'year', 'month'],
      dtype='object')

In [44]:
unique_users = train.user_id.unique()


array([     12,      93,     501, ..., 1198099, 1198173, 1198182], dtype=int64)

In [45]:
len(unique_users)

1198786

In [15]:
sel_user_ids = [unique_users[i] for i in sorted( random.sample( range(len(unique_users)) , 10000 ) ) ]

In [16]:
sel_train = train[train.user_id.isin(sel_user_ids)]

In [17]:
# t1 = sel_train[((sel_train.year == 2013) | ((sel_train.year == 2014) & (sel_train.month < 8)))]
# 

In [18]:
# t2 = sel_train[((sel_train.year == 2014) & (sel_train.month >= 8))]
# t2 = t2[t2.is_booking == True]

In [19]:
### this is where you can swtich t1 with train and t2 with test

In [62]:
t1 = train # too big!
# t1 = sel_train
t2 = test # necessary

In [21]:
most_common_clusters = list(train.hotel_cluster.value_counts().head().index)

In [22]:
most_common_clusters

[91, 41, 48, 64, 65]

In [23]:
predictions = [most_common_clusters for i in range(t2.shape[0])]

In [46]:
# target = [[l] for l in t2["hotel_cluster"]]
# metrics.mapk(target, predictions, k=5)

In [ ]:
## Vik tutorial-maker gets 0.058020770920711007 here - quite a difference - but we *do* have a random sample of 10,000

In [ ]:
# list(train.columns).index('hotel_cluster')

In [ ]:
# tr_hc = pd.DataFrame(train["hotel_cluster"])


In [ ]:
# features = ['site_name', 'posa_continent', 'user_location_country',
#        'user_location_region', 'user_location_city',
#        'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
#        'channel', 'srch_adults_cnt', 'srch_children_cnt',
#        'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id',
#        'is_booking', 'cnt', 'hotel_continent', 'hotel_country', 'hotel_market',
#        'year', 'month']
# # WARNING V MEMORY INTENSIVE - Didn't work on 16GB win64 - still crashed with 8Gb SSD swap
# # train.corr()["hotel_cluster"]
# for ft in features:
#     print(ft,tr_hc.corrwith(train[ft]))

In [25]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
dest_small = pca.fit_transform(destinations[["d{0}".format(i + 1) for i in range(149)]])
dest_small = pd.DataFrame(dest_small)
dest_small["srch_destination_id"] = destinations["srch_destination_id"]



In [26]:
# dest_small

In [27]:
def calc_fast_features(df):
#     df["date_time"] = pd.to_datetime(df["date_time"])
#     df["srch_ci"] = pd.to_datetime(df["srch_ci"], format='%Y-%m-%d', errors="coerce")
#     df["srch_co"] = pd.to_datetime(df["srch_co"], format='%Y-%m-%d', errors="coerce")
    
    props = {}
    for prop in ["month", "day", "hour", "minute", "dayofweek", "quarter"]:
        props[prop] = getattr(df["date_time"].dt, prop)
    
    carryover = [p for p in df.columns if p not in ["date_time", "srch_ci", "srch_co"]]
    for prop in carryover:
        props[prop] = df[prop]
    
    date_props = ["month", "day", "dayofweek", "quarter"]
    for prop in date_props:
        props["ci_{0}".format(prop)] = getattr(df["srch_ci"].dt, prop)
        props["co_{0}".format(prop)] = getattr(df["srch_co"].dt, prop)
    props["stay_span"] = (df["srch_co"] - df["srch_ci"]).astype('timedelta64[h]')
        
    ret = pd.DataFrame(props)
    
    ret = ret.join(dest_small, on="srch_destination_id", how='left', rsuffix="dest") ## this is where it crashed with full train
    ret = ret.drop("srch_destination_iddest", axis=1)
    return ret



In [63]:
df = t1

In [64]:
# props = {}
date_time_props = ["month", "day", "hour", "minute", "dayofweek", "quarter"]
for prop in date_time_props:
    df[prop] = getattr(df["date_time"].dt, prop)
    
df.drop("date_time", axis=1)

In [67]:
# carryover = [p for p in df.columns if p not in ["date_time", "srch_ci", "srch_co"]]
# for prop in carryover:
#     props[prop] = df[prop]

In [68]:
date_props = ["month", "day", "dayofweek", "quarter"]
for prop in date_props:
    df["ci_{0}".format(prop)] = getattr(df["srch_ci"].dt, prop)
    df["co_{0}".format(prop)] = getattr(df["srch_co"].dt, prop)
    
    

In [ ]:
df["stay_span"] = (df["srch_co"] - df["srch_ci"]).astype('timedelta64[h]')

In [55]:
# ret = pd.DataFrame(props)

In [56]:
df = df.join(dest_small, on="srch_destination_id", how='left', rsuffix="dest") ## this is where it crashed with full trai

In [57]:
df = df.drop("srch_destination_iddest", axis=1)

In [58]:
# df = ret

In [59]:
# df = calc_fast_features(t1)
df.fillna(-1, inplace=True)

In [47]:
df

,channel,ci_day,ci_dayofweek,ci_month,ci_quarter,cnt,co_day,co_dayofweek,co_month,co_quarter,...,srch_rm_cnt,stay_span,user_id,user_location_city,user_location_country,user_location_region,year,0,1,2
2394,0,5.0,1.0,3.0,1.0,1,13.0,2.0,3.0,1.0,...,1,192.0,10643,16460,205,135,2013,0.166435,0.030140,-0.103943
2395,3,5.0,1.0,3.0,1.0,1,13.0,2.0,3.0,1.0,...,1,192.0,10643,16460,205,135,2013,0.305514,0.049351,-0.050840
2396,3,8.0,1.0,1.0,1.0,1,15.0,1.0,1.0,1.0,...,1,168.0,10643,16460,205,135,2013,0.305514,0.049351,-0.050840
2397,9,5.0,1.0,3.0,1.0,1,13.0,2.0,3.0,1.0,...,1,192.0,10643,16460,205,135,2013,0.000411,0.145256,0.011122
2398,9,5.0,1.0,3.0,1.0,1,13.0,2.0,3.0,1.0,...,1,192.0,10643,16460,205,135,2013,-0.597606,0.243432,-0.047467
2399,2,5.0,1.0,3.0,1.0,1,13.0,2.0,3.0,1.0,...,1,192.0,10643,16460,205,135,2013,-0.788613,0.367128,0.169239
2400,2,5.0,1.0,3.0,1.0,1,13.0,2.0,3.0,1.0,...,1,192.0,10643,16460,205,135,2013,-0.788613,0.367128,0.169239
2401,9,6.0,2.0,3.0,1.0,1,13.0,2.0,3.0,1.0,...,1,168.0,10643,16460,205,135,2013,0.000411,0.145256,0.011122
2402,1,21.0,3.0,3.0,1.0,1,26.0,1.0,3.0,1.0,...,1,120.0,10643,5703,3,50,2013,0.560557,-0.017841,0.038934
2403,1,22.0,4.0,3.0,1.0,1,27.0,2.0,3.0,1.0,...,1,120.0,10643,5703,3,50,2013,0.560557,-0.017841,0.038934


In [30]:
# # Random forest, classifying to 100 categories
# predictors = [c for c in df.columns if c not in ["hotel_cluster"]]
# # from sklearn import cross_validation
# # from sklearn.ensemble import RandomForestClassifier

# clf = RandomForestClassifier(n_estimators=10, min_weight_fraction_leaf=0.1)
# scores = cross_validation.cross_val_score(clf, df[predictors], df['hotel_cluster'], cv=3)
# scores

In [31]:
# # Random forest, but with only binary classification
# all_probs = []
# unique_clusters = df["hotel_cluster"].unique()


In [32]:
# for cluster in unique_clusters:
#     df["target"] = 1
#     #df[df['A'] > 2]['B'] = new_val
#     #df.loc[df['A'] > 2, 'B'] = new_val
#     df.loc[df["hotel_cluster"] != cluster, "target"] = 0
# #     df["target"][df["hotel_cluster"] != cluster] = 0
#     predictors = [col for col in df if col not in ['hotel_cluster', "target"]]
#     probs = []
#     cv = KFold(len(df["target"]), n_folds=2)
#     clf = RandomForestClassifier(n_estimators=10, min_weight_fraction_leaf=0.1)
#     for i, (tr, te) in enumerate(cv):
#         clf.fit(df[predictors].iloc[tr], df["target"].iloc[tr])
#         preds = clf.predict_proba(df[predictors].iloc[te])
#         probs.append([p[1] for p in preds])
#     full_probs = chain.from_iterable(probs)
#     all_probs.append(list(full_probs))



In [33]:
# prediction_frame = pd.DataFrame(all_probs).T
# prediction_frame.columns = unique_clusters
# def find_top_5(row):
#     return list(row.nlargest(5).index)

# preds = []
# for index, row in prediction_frame.iterrows():
#     preds.append(find_top_5(row))



In [34]:
# metrics.mapk([[l] for l in t2["hotel_cluster"]], preds, k=5)

In [35]:
def make_key(items):
    return "_".join([str(i) for i in items])

match_cols = ["srch_destination_id"]
cluster_cols = match_cols + ['hotel_cluster']
groups = t1.groupby(cluster_cols)
top_clusters = {}
for name, group in groups:
    clicks = len(group.is_booking[group.is_booking == False])
    bookings = len(group.is_booking[group.is_booking == True])
    
    score = bookings + .15 * clicks
    
    clus_name = make_key(name[:len(match_cols)])
    if clus_name not in top_clusters:
        top_clusters[clus_name] = {}
    top_clusters[clus_name][name[-1]] = score

In [36]:

cluster_dict = {}
for n in top_clusters:
    tc = top_clusters[n]
    top = [l[0] for l in sorted(tc.items(), key=operator.itemgetter(1), reverse=True)[:5]]
    cluster_dict[n] = top

In [37]:
preds = []
for index, row in t2.iterrows():
    key = make_key([row[m] for m in match_cols])
    if key in cluster_dict:
        preds.append(cluster_dict[key])
    else:
        preds.append([])

In [38]:
# metrics.mapk([[l] for l in t2["hotel_cluster"]], preds, k=5)

In [39]:
match_cols = ['user_location_country', 'user_location_region', 'user_location_city', 'hotel_market', 'orig_destination_distance']

groups = t1.groupby(match_cols)
    
def generate_exact_matches(row, match_cols):
    index = tuple([row[t] for t in match_cols])
    try:
        group = groups.get_group(index)
    except Exception:
        return []
    clus = list(set(group.hotel_cluster))
    return clus

exact_matches = []
for i in range(t2.shape[0]):
    exact_matches.append(generate_exact_matches(t2.iloc[i], match_cols))

In [40]:
def f5(seq, idfun=None): 
    if idfun is None:
        def idfun(x): return x
    seen = {}
    result = []
    for item in seq:
        marker = idfun(item)
        if marker in seen: continue
        seen[marker] = 1
        result.append(item)
    return result
    
full_preds = [f5(exact_matches[p] + preds[p] + most_common_clusters)[:5] for p in range(len(preds))]
# metrics.mapk([[l] for l in t2["hotel_cluster"]], full_preds, k=5)

In [41]:
write_p = [" ".join([str(l) for l in p]) for p in full_preds]
write_frame = ["{0},{1}".format(t2["id"][i], write_p[i]) for i in range(len(full_preds))]
write_frame = ["id,hotel_cluster"] + write_frame
with open("predictions.csv", "w+") as f:
    f.write("\n".join(write_frame))

In [43]:
import os, datetime

sub_dir = os.path.join( r"..\expedia_submissions",
                       datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S') )

os.makedirs(sub_dir)
filename = "predictions.csv"
with open( os.path.join( sub_dir, filename ), "w+" ) as f:
    f.write( "\n".join(write_frame) )